In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
from tensorflow.keras import layers ,activations,models,preprocessing,utils

In [4]:
data="chatbot dataset.txt"

Creating the data for the input_put and target 

In [5]:
input_texts=[]
target_texts=[]
with open(data) as f:
    lines=f.read().split('\n')
for line in lines[:min(600,len(lines) -1)]:
    input_text=line.split('\t')[0]
    target_text=line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [6]:
print("Type of input text :",type(input_text))
print("Type of target_text:",type(target_texts))

Type of input text : <class 'str'>
Type of target_text: <class 'list'>


converting the list in pandas dataframe since input_text,target_text are both are type of list

In [7]:
zippedlist=list(zip(input_texts,target_texts))
lines=pd.DataFrame(zippedlist,columns=['input','output'])


In [8]:
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


Prepareing the data for the encoder


In [9]:
input_line=list()
for line in lines.input:
    input_line.append(line)

In [10]:
print("Input lines are :",input_line)

Input lines are : ['What are your interests', 'What are your favorite subjects', 'What are your interests', 'What is your number', 'What is your number', 'What is your favorite number', 'What can you eat', "Why can't you eat food", 'What is your location', 'What is your location', 'Where are you from', 'Where are you', 'Do you have any brothers', 'Do you have any brothers', 'Who is your father', 'Who is your mother', 'Who is your boss', 'What is your age', 'What is your age', 'what is the illuminati', 'what is the illuminatti', 'what is the illuminatti', 'what is vineland', 'What is Illuminatus', 'What is Illuminatus', 'who wrote vineland', 'who is bilbo baggins', 'who is geoffrey chaucer', 'who is piers anthony', 'have you read plato', 'have you read frankenstein', 'have you ever read a book', 'have you ever read a book', 'have you ever read a book', 'have you read many books', 'have you read homer', 'ray bradbury', 'what is mind children', 'william gibson', 'william gibson', 'holden 

what is Encoder and decoder?




The encoder-decoder model is a way of using recurrent neural networks for sequence-to-sequence prediction problems.

It was initially developed for machine translation problems, although it has proven successful at related sequence-to-sequence prediction problems such as text summarization and question answering.

The approach involves two recurrent neural networks, one to encode the input sequence, called the encoder, and a second to decode the encoded input sequence into the target sequence called the decoder

Data preprocessing for encoder

In [11]:
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_line)
tokenizer_input=tokenizer.texts_to_sequences(input_line)

lenght_list=list()
for token_seq in tokenizer_input:
    lenght_list.append(len(token_seq))
max_input_length=np.array(lenght_list).max()
print("Input max length is {}".format(max_input_length))
padded_input_lines=preprocessing.sequence.pad_sequences(tokenizer_input,maxlen=max_input_length,padding='post')
encoder_input_data=np.array(padded_input_lines)
print("Encoder input data shape {}".format(encoder_input_data.shape))

input_word_dict=tokenizer.word_index
nump_input_token=len(input_word_dict)+1
print("Number of Input token={}".format(nump_input_token))

Input max length is 22
Encoder input data shape (566, 22)
Number of Input token=518


In [12]:
input_word_dict

{'you': 1,
 'is': 2,
 'are': 3,
 'what': 4,
 'a': 5,
 'do': 6,
 'me': 7,
 'tell': 8,
 'the': 9,
 'joke': 10,
 'not': 11,
 'your': 12,
 'to': 13,
 'how': 14,
 'who': 15,
 'can': 16,
 'of': 17,
 'it': 18,
 'like': 19,
 'i': 20,
 'have': 21,
 'computer': 22,
 'die': 23,
 'eat': 24,
 'stock': 25,
 'about': 26,
 'get': 27,
 'market': 28,
 'hal': 29,
 'ever': 30,
 'in': 31,
 'know': 32,
 'gossip': 33,
 'gossips': 34,
 'robots': 35,
 'will': 36,
 'read': 37,
 'an': 38,
 'when': 39,
 'hi': 40,
 'favorite': 41,
 'robot': 42,
 'up': 43,
 'much': 44,
 'sense': 45,
 'why': 46,
 'should': 47,
 'feel': 48,
 'make': 49,
 'mad': 50,
 'bad': 51,
 'was': 52,
 'immortal': 53,
 'soccer': 54,
 'going': 55,
 'did': 56,
 'does': 57,
 'be': 58,
 'money': 59,
 'guns': 60,
 'making': 61,
 'baseball': 62,
 'play': 63,
 "what's": 64,
 'number': 65,
 'any': 66,
 'wrote': 67,
 'frank': 68,
 'herbert': 69,
 'asimov': 70,
 'anybody': 71,
 'wavelength': 72,
 'far': 73,
 'sad': 74,
 'nice': 75,
 'that': 76,
 'lie': 77,

In [13]:
encoder_input_data

array([[  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       ...,
       [498, 499,   9, ...,   0,   0,   0],
       [  9, 207, 208, ...,   0,   0,   0],
       [510, 192,   5, ...,   0,   0,   0]], dtype=int32)

Prepare the input data for the decoder

In [14]:
output_lines=list()
for line in lines.output:
    output_lines.append('<START>'+line+"<END>")

In [15]:
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(output_lines)
token_output=tokenizer.texts_to_sequences(output_lines)

In [16]:
length_list=list()
for token_seq in token_output:
    length_list.append(len(token_seq))
max_output=np.array(length_list).max()
print("Output length{}".format(max_output))

padded_output=preprocessing.sequence.pad_sequences(token_output,maxlen=max_output,padding='post')
decoder_input_data=np.array(padded_output)
print("Decoder input data shape :{}".format(decoder_input_data.shape))
output_worddic=tokenizer.word_index
num_out_token=len(output_worddic)+1
print("Number of Output tokens= {}".format(num_out_token))

Output length74
Decoder input data shape :(566, 74)
Number of Output tokens= 1692


Preparing the data for the decoder

In [17]:
decoder_target_data=list()
for token_seq in token_output:
    decoder_target_data.append(token_seq[1:])
padded_output_lines=preprocessing.sequence.pad_sequences(decoder_target_data,maxlen=max_output,padding='post')
onehot_output_lines=utils.to_categorical(padded_output_lines,num_out_token)
decoder_target_data=np.array(onehot_output_lines)
print("Decoder target shape {}".format(decoder_target_data.shape))

Decoder target shape (566, 74, 1692)


Defining the model

(state_c)  Cell state: Long term memory of the model, only part of LSTM models


(state_h)   Hidden state: Working memory, part of LSTM and RNN models

What is embedding layer used for?



Embedding layer enables us to convert each word into a fixed length vector of defined size. The resultant vector is a dense one with having real values instead of just 0's and 1's. The fixed length of word vectors helps us to represent words in a better way along with reduced dimensions.

In [18]:
encoder_inputs=tf.keras.layers.Input(shape=(None,))
encoder_embedding=tf.keras.layers.Embedding(nump_input_token,256,mask_zero=True)(encoder_inputs)
encoder_outputs,state_h,state_c=tf.keras.layers.LSTM(256,return_state=True,recurrent_dropout=0.2,dropout=0.2)(encoder_embedding)
encoder_states=[state_h,state_c]

####################################################

decoder_inputs=tf.keras.layers.Input(shape=(None,))
decoder_embedding=tf.keras.layers.Embedding(num_out_token,256,mask_zero=True)(decoder_inputs)
decoder_lstm=tf.keras.layers.LSTM(256,return_state=True,return_sequences=True,recurrent_dropout=0.2,dropout=0.2)
decoder_outputs,_,_=decoder_lstm(decoder_embedding,initial_state=encoder_states)

decoder_dense=tf.keras.layers.Dense(num_out_token,activation=tf.keras.activations.softmax)
output=decoder_dense(decoder_outputs)

model=tf.keras.models.Model([encoder_inputs,decoder_inputs],output)

model.compile(optimizer=tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=['accuracy'])

model.summary()


2023-03-03 12:40:01.346100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-03 12:40:01.346134: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-03 12:40:01.346162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2023-03-03 12:40:01.346434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    132608      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    433152      ['input_2[0][0]']                
                                                                                              

In [19]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=124,epochs=250)


Epoch 1/250
5/5 [==============================] - 13s 2s/step - loss: 7.4266 - accuracy: 0.0744
Epoch 2/250
5/5 [==============================] - 9s 2s/step - loss: 7.3210 - accuracy: 0.1067
Epoch 3/250
5/5 [==============================] - 9s 2s/step - loss: 6.6576 - accuracy: 0.0938
Epoch 4/250
5/5 [==============================] - 9s 2s/step - loss: 5.9621 - accuracy: 0.0771
Epoch 5/250
5/5 [==============================] - 10s 2s/step - loss: 5.8536 - accuracy: 0.0707
Epoch 6/250
5/5 [==============================] - 9s 2s/step - loss: 5.7598 - accuracy: 0.1076
Epoch 7/250
5/5 [==============================] - 9s 2s/step - loss: 5.7120 - accuracy: 0.1392
Epoch 8/250
5/5 [==============================] - 9s 2s/step - loss: 5.6816 - accuracy: 0.1380
Epoch 9/250
5/5 [==============================] - 9s 2s/step - loss: 5.6518 - accuracy: 0.1282
Epoch 10/250
5/5 [==============================] - 9s 2s/step - loss: 5.6219 - accuracy: 0.1416
Epoch 11/250
5/5 [===================

5/5 [==============================] - 3s 628ms/step - loss: 3.0620 - accuracy: 0.3829
Epoch 84/250
5/5 [==============================] - 4s 725ms/step - loss: 3.0260 - accuracy: 0.3852
Epoch 85/250
5/5 [==============================] - 4s 737ms/step - loss: 3.0046 - accuracy: 0.3900
Epoch 86/250
5/5 [==============================] - 3s 627ms/step - loss: 2.9612 - accuracy: 0.3953
Epoch 87/250
5/5 [==============================] - 3s 621ms/step - loss: 2.9499 - accuracy: 0.3980
Epoch 88/250
5/5 [==============================] - 3s 654ms/step - loss: 2.9187 - accuracy: 0.3978
Epoch 89/250
5/5 [==============================] - 3s 591ms/step - loss: 2.8883 - accuracy: 0.4028
Epoch 90/250
5/5 [==============================] - 3s 656ms/step - loss: 2.8649 - accuracy: 0.4069
Epoch 91/250
5/5 [==============================] - 3s 653ms/step - loss: 2.8377 - accuracy: 0.4109
Epoch 92/250
5/5 [==============================] - 3s 593ms/step - loss: 2.8071 - accuracy: 0.4155
Epoch 93/250


5/5 [==============================] - 3s 683ms/step - loss: 1.2274 - accuracy: 0.7431
Epoch 165/250
5/5 [==============================] - 3s 599ms/step - loss: 1.2127 - accuracy: 0.7501
Epoch 166/250
5/5 [==============================] - 4s 678ms/step - loss: 1.1993 - accuracy: 0.7530
Epoch 167/250
5/5 [==============================] - 3s 626ms/step - loss: 1.1890 - accuracy: 0.7539
Epoch 168/250
5/5 [==============================] - 3s 675ms/step - loss: 1.1748 - accuracy: 0.7571
Epoch 169/250
5/5 [==============================] - 4s 714ms/step - loss: 1.1532 - accuracy: 0.7640
Epoch 170/250
5/5 [==============================] - 3s 598ms/step - loss: 1.1448 - accuracy: 0.7641
Epoch 171/250
5/5 [==============================] - 3s 681ms/step - loss: 1.1314 - accuracy: 0.7689
Epoch 172/250
5/5 [==============================] - 3s 626ms/step - loss: 1.1160 - accuracy: 0.7714
Epoch 173/250
5/5 [==============================] - 3s 626ms/step - loss: 1.1071 - accuracy: 0.7758
Epoc

5/5 [==============================] - 3s 679ms/step - loss: 0.5148 - accuracy: 0.9063
Epoch 246/250
5/5 [==============================] - 4s 708ms/step - loss: 0.5143 - accuracy: 0.9077
Epoch 247/250
5/5 [==============================] - 4s 710ms/step - loss: 0.4986 - accuracy: 0.9120
Epoch 248/250
5/5 [==============================] - 4s 676ms/step - loss: 0.5016 - accuracy: 0.9083
Epoch 249/250
5/5 [==============================] - 4s 673ms/step - loss: 0.4959 - accuracy: 0.9122
Epoch 250/250
5/5 [==============================] - 4s 756ms/step - loss: 0.4936 - accuracy: 0.9094


In [20]:
model.save('sch1.h5')

In [21]:
#model=load_model('sch1.h5')

Inference models

In [22]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [23]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [28]:
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_worddic['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_worddic.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', ' ') )
    print()

User: what is your number
1/1 [==============================] - 0s 17ms/step
Bot: i am quite fond of the number 42 

User: hi
1/1 [==============================] - 0s 17ms/step
Bot: hello 

User: hello
1/1 [==============================] - 0s 17ms/step
Bot: hi 

User: what are your favorite
1/1 [==============================] - 0s 17ms/step
Bot: i couldn't have said it better myself 

User: what are your favorite subjects
1/1 [==============================] - 0s 17ms/step
Bot: my favorite subjects include robotics computer science and natural language processing 

User: end


KeyError: 'end'